# <a href="https://mipt-stats.gitlab.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Задание 5

**Правила:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`.
* Дедлайн **10 апреля в 23:00**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb` и все фотографии, если пишете теоретическую часть от руки.
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него.

-----

*Замечания.* Теоретические решения можно оформить
* в $\LaTeX$-формате в ноутбуке;
* написать от руки и прикрепить к ноутбуку;
* написать от руки и выслать боту.  

Во втором случае также **важно** "вшить" фото в ноутбук. Сделать это можно с помощью Edit -> Insert Image в Jupyter или с помощью кнопки "Вставить изображение" в Colab. Следите за размером итогового файла.

Фотографии принимаются только в хорошем качестве, **следите за освещением и почерком**. На фотографиях также указывайте номера задач.

-----

In [183]:
import numpy as np
import pandas as pd
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns

red = '#FF3300'
blue = '#0099CC'
green = '#00CC66'

from statsmodels.sandbox.stats.multicomp import multipletests
#from tqdm.notebook import tqdm

%matplotlib inline


# Теоретическая часть

## p-value (основной поток)

### Задача 1. 
Для всех пунктов задач 3 и 4 прошлого домашнего задания выпишите формулу для p-value в виде кода на `scipy`, где $t$ - реализация статистики вашего критерия, т.е. $t = T(x), x$ &mdash; реализация выборки, $T(X)$ &mdash; статистика.

>* 3а. $p(t) =$ `sps.norm(theta_0, 1 / n).cdf(t)`. 
>* 3б. $p(t) =$ `sps.norm(theta_0, 1 / n).sf(t)`.
>* 4а. $p(t) =$ `sps.gamma(a=2, scale=7/6).sf(t)` $=0.022$. 
>* 4б. $p(t) =$ `sps.gamma(a=3, scale=44/5).cdf(t)` $=0.041$.

Для задачи 4 также посчитайте численные значения для обоих случаев p-value для данного в задаче существа. Какие гипотезы отклоняются? 

>Отклоняются обе гипотезы, но первая с большей уверенностью.

Вычисления можно выполнить в Питоне по приведенным вами формулам. 

## Множественная проверка гипотез (основной поток)


### Задача 2.
 Пусть $X_1, ...,  X_n$ &mdash; выборка из неизвестного распределения $\mathsf{P}$. Для проверки гипотезы $\mathsf{H}_0\ vs.\ \mathsf{H}_1$ было решено использовать три различных критерия. Соответствующие p-value равны 0.00001, 0.7361, 0.0482. Какое должно быть принято решение об отвержении гипотезы $\mathsf{H}_0$ на уровне значимости 0.05? *Подсказка: используйте любой подходящий метод МПГ, далее делайте вывод об отвержении/не отвержении, поясните свой вывод.*




>Так как мы ничего не знаем о зависимости статистик, то пременим метод Холма, затем применим метод Бонферони для сравнения. Далее попробуем зайти с дргой сторны и ограничить FDR с помощью метода Бенджамини-Иекутели.

In [184]:
p_value = np.array([0.00001, 0.0361, 0.0482])

In [185]:
from statsmodels import stats as st

b = st.multitest.multipletests(pvals=p_value, alpha=0.05, method='bonferroni', is_sorted=False, returnsorted=False)
h = st.multitest.multipletests(pvals=p_value, alpha=0.05, method='holm', is_sorted=False, returnsorted=False)
by = st.multitest.multipletests(pvals=p_value, alpha=0.05, method='fdr_by', is_sorted=False, returnsorted=False)

methods = {1: [b[0][0], h[0][0], by[0][0]],
           2: [b[0][1], h[0][1], by[0][1]],
           3: [b[0][2], h[0][2], by[0][2]],
           'pv\'1': [b[1][0], h[1][0], by[1][0]],
           'pv\'2': [b[1][1], h[1][1], by[1][1]],
           'pv\'3': [b[1][2], h[1][2], by[1][2]]}

pd.DataFrame(methods, index=['bonferroni', 'holm', 'fdr_by'])

,1,2,3,pv'1,pv'2,pv'3
bonferroni,True,False,False,0.000030,0.108300,0.144600
holm,True,False,False,0.000030,0.072200,0.072200
fdr_by,True,False,False,0.000055,0.088367,0.088367


>**Вывод:** Как видно, все методы выдали одинаковый результат: два из трх критереев показывают, что гипотезу овергать не стоит. Так как Метод Холма является наиболее мощным, то меньшие значения $\text{p-value}$ при ограничении FWER мы уже получить не должны, а значит, два критерия и правда не отвергают гипотезу. Аналогично метод Бенджамини-Иекутели явлляется наиболее мощным при ограничении FDR, что говорит о том, что первый критерий и правда отвергает гипотезу. Так как мы не отвергли гипотезу чаще при множественной проверке, то, наверно, её не стоит отвергать. Стоит попробовать применить ещё какие-нибудь критерии (возможно, первый был слишком сильным или два других слишком слабыми).

# Практическая часть

## Множественная проверка гипотез


### Задача 3 (все потоки).
Проведены эксперименты для оценки эффективности нескольких препаратов для снижения послеоперационной тошноты. Результаты экспериментов приведены в таблице ниже. При проведении эксперимента пациенты делились на группы случайным образом.

    
                            Количество пациентов  Количество случаев возникновения тошноты

		Плацебо                80                    45 

		Хлорпромазин           75                    26 
    
		Дименгидринат          85                    52 
    
		Пентобарбитал (100 мг) 67                    35 
    
		Пентобарбитал (150 мг) 85                    37 
    

 Проведите сравнение каждого препарата по эффективности по отношению к плацебо c использованием критерия Вальда (см. лекцию 5 и последующие). Какие методы МПГ, контролирующие FWER и FDR, можно использовать в данной задаче? Какие ответы можно получить для этих методов? В каждом случае приведите значения статистики критерия Вальда, p-value и скорректированные p-value. Поясните смысл p-value и множественной проверки гипотез в данной задаче. Оформите решение структурированно. 

In [186]:
data = pd.DataFrame([["Плацебо", 80, 45],
    ["Хлорпромазин", 75, 26],
    ["Дименгидринат", 85, 52],
    ["Пентобарбитал (100 мг)", 67, 35],
    ["Пентобарбитал (150 мг)", 85, 37]])

data.columns = ["Название", "Количество пациентов", "Количество случаев возникновения тошноты"]

In [187]:
data

,Название,Количество пациентов,Количество случаев возникновения тошноты
0,Плацебо,80,45
1,Хлорпромазин,75,26
2,Дименгидринат,85,52
3,Пентобарбитал (100 мг),67,35
4,Пентобарбитал (150 мг),85,37


>**Критерий Вальда.**
>
>Пусть $X_{i}=(X_{i1},...,X_{in_i})$ &mdash; выборка для $i$-го эксперимента. Псть $X_i\sim Bern(p_i)$. Пусть $Y=(Y_{1},...,Y_{n_p})$ &mdash; выборка для плацебо, распередлённая как $Bern(p_p)$. Тогда будем проверять  гипотезы $H_{0i}$: $p_i=p_p$ &mdash; отсутствие эффекта, и $H_{1i}$: $p_i<p_p$ &mdash; эффект препарата сильнее.
>
>Для этого воспользуемся критерием Вальда: $S_i=\{W(X_i, Y)<z_{\alpha}\}$, где $W(X_i, Y)=\frac{\widehat{p_i}-\widehat{p_p}}{\widehat{\sigma_i}}$. Оценки параметров равны $\widehat{p_i}=\overline{X_i}$, $\widehat{p_p}=\overline{Y}$, $\widehat{\sigma_i}=\sqrt{\frac{\widehat{p_i}(1-\widehat{p_i})}{n_i}+\frac{\widehat{p_p}(1-\widehat{p_p})}{n_p}}$.
>
>Для уровня значимости $\alpha=0.05$ получим $z_{\alpha}\approx-1.64$.

In [188]:
def W(x_sum, x_size, y_sum, y_size):
    p1 = x_sum / x_size
    p2 = y_sum / y_size
    sigma = (p1 * (1 - p1) / x_size + p2 * (1 - p2) / y_size) ** 0.5
    return (p1 - p2) / sigma

In [189]:
alpha = 0.05
z = sps.norm.ppf(alpha)

In [190]:
y_size, y_sum = data.T[0].values[1:]

x_size, x_sum = np.array(data.T[data.T.columns[1:]].values[1:], dtype=int)

wald = W(x_sum, x_size, y_sum, y_size)

pd.DataFrame(zip(data['Название'].values[1:], wald, wald < z), columns=['Название', 'Критеррий Вальда', 'Отвергаем?'])

,Название,Критеррий Вальда,Отвергаем?
0,Хлорпромазин,-2.764364,True
1,Дименгидринат,0.642987,False
2,Пентобарбитал (100 мг),-0.486428,False
3,Пентобарбитал (150 мг),-1.646605,True


>$\textbf{p-value}.$
>
>Рассчитаем $\text{p-value}$ по формуле $\text{p-value}=\mathbf{P}\left(\frac{\widehat{p_i}-\widehat{p_p}}{\widehat{\sigma_i}}\leq t\right)=$ `sps.norm.cdf(t)`, где $t$ &mdash; реализация статистики.

In [191]:
pv = sps.norm.cdf(wald)

pd.DataFrame(zip(data['Название'].values[1:], pv, pv < alpha), columns=['Название', 'p-value', 'Отвергаем?'])

,Название,p-value,Отвергаем?
0,Хлорпромазин,0.002852,True
1,Дименгидринат,0.739884,False
2,Пентобарбитал (100 мг),0.313332,False
3,Пентобарбитал (150 мг),0.049820,True


>Как видно из таблицы, гипотеза для последнего препарата отвергнута с очень плохой уверенностью.

>**Множественная проверка гипотез.**
>
>Так как по условию пациенты делились на группы случайным образом, то выборки являются независимыми, то для контроля FWER можем воспользоваться методом Шидака-Холма, а для контроля FDR &mdash; методом Бенджамини-Хохберга. Все методы как минимум должны выдать повышенное значение $\text{p-value}$, но у метода Шидака-Холма они должны быть выше.

Метод Шидака-Холма

In [192]:
hs = st.multitest.multipletests(pvals=pv, alpha=alpha, method='hs', is_sorted=False, returnsorted=False)

pd.DataFrame(zip(data['Название'].values[1:], pv, hs[1], hs[0]), columns=['Название', 'p-value', 'Скорректированный p-value', 'Отвергаем?'])

,Название,p-value,Скорректированный p-value,Отвергаем?
0,Хлорпромазин,0.002852,0.011358,True
1,Дименгидринат,0.739884,0.739884,False
2,Пентобарбитал (100 мг),0.313332,0.528487,False
3,Пентобарбитал (150 мг),0.049820,0.142137,False


Метод Бенджамини-Хохберга

In [193]:
bh = st.multitest.multipletests(pvals=pv, alpha=alpha, method='fdr_bh', is_sorted=False, returnsorted=False)

pd.DataFrame(zip(data['Название'].values[1:], pv, bh[1], bh[0]), columns=['Название', 'p-value', 'Скорректированный p-value', 'Отвергаем?'])

,Название,p-value,Скорректированный p-value,Отвергаем?
0,Хлорпромазин,0.002852,0.011407,True
1,Дименгидринат,0.739884,0.739884,False
2,Пентобарбитал (100 мг),0.313332,0.417776,False
3,Пентобарбитал (150 мг),0.049820,0.099639,False


>Как видно, метод Беджамини-Хохберга выдал меньшие значения скоректированных $\text{p-value}$, что и ожидалось, так как он контролирует FDR.

>**Вывод:** С помощью критерия Вальда мы смогли отвергнуть две гипотезы, однако значения $\text{p-value}$ говорят о том, что с хорошей уверенностью можно отвергнуть только гипотезу для Хлорпромазина. МПГ же говорит нам, что отвергать можно лишь её. В этом случае МПГ, возможно, даёт нам какое-то представление о практической значимости результата, так как повышает $\text{p-value}$, что разрешает отвергать более экстремальные выборки, чем предполагалось изначально (что и повлияло на неотвержение гипотезы о Пентобарбитал (150 мг) в силу слишком близкого к $\alpha$ $\text{p-value}$).